<a href="https://colab.research.google.com/github/AUMANSH/Data-Science-Projects/blob/main/CNN_Chest_X_Ray_Images_(Pneumonia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [00:20<00:00, 311MB/s]
100% 2.29G/2.29G [00:20<00:00, 122MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/chest-xray-pneumonia.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
train = keras.utils.image_dataset_from_directory(
    directory='/content/chest_xray/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
)

test = keras.utils.image_dataset_from_directory(
    directory='/content/chest_xray/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [ ]:
#normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train.map(process)
test_ds = test.map(process)

print(train_ds)
print(test_ds)
#train_ds = train_ds/255
#validation_ds = validation_ds/255

<_MapDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_MapDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='valid', strides=(2, 2), input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), padding='valid', strides=2))

model.add(Conv2D(32, (3, 3), activation='relu', padding='valid', strides=(2, 2)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), padding='valid', strides=2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 127, 127, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 127, 127, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 31, 31, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 31, 31, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 13, 13, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 13, 13, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 648,225 (2.47 MB)

 Trainable params: 647,777 (2.47 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history1 = model.fit(train_ds, epochs=10,batch_size=64,validation_data=test_ds)

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 41s 204ms/step - accuracy: 0.8634 - loss: 0.5914 - val_accuracy: 0.6250 - val_loss: 9.1312
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 36s 218ms/step - accuracy: 0.9582 - loss: 0.1095 - val_accuracy: 0.6250 - val_loss: 5.7027
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 39s 209ms/step - accuracy: 0.9687 - loss: 0.0913 - val_accuracy: 0.8045 - val_loss: 0.8060
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 40s 206ms/step - accuracy: 0.9778 - loss: 0.0637 - val_accuracy: 0.7436 - val_loss: 0.9284
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 41s 207ms/step - accuracy: 0.9763 - loss: 0.0672 - val_accuracy: 0.6699 - val_loss: 3.0541
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 36s 224ms/step - accuracy: 0.9778 - loss: 0.0643 - val_accuracy: 0.7692 - val_loss: 0.8592
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 38s 208ms/step - accuracy: 0.9818 - loss: 0.0518 - val_accuracy: 0.6346 - val_loss: 2.3164
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 40s 200ms/step - accuracy: 0.9825 - loss: 0

In [ ]:
y_pred = model.predict(test_ds)
y_pred = (y_pred > 0.5).astype("int32")

20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 183ms/step


In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Extract true labels
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"Accuracy on validation set: {acc * 100:.2f}%")

Accuracy on validation set: 53.21%
